#### Portfolio Risk Analysis
long-short corporate bond portfolio

taking only issuer credit risk, hedging with IR, FX, etc

In [8]:
%%capture
%pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org matplotlib, pandas, scipy

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import scipy.cluster.hierarchy as sch
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import LinearOperator

In [10]:
df_port_wgt = pd.read_csv('portfolio_weights.csv', index_col=['ID'])
df_mkt_wgt = pd.read_csv('market_portfolio_weights.csv', index_col=['ID'])
print(len(df_port_wgt), len(df_mkt_wgt))

240 11797


In [11]:
df_factor_exp = pd.read_csv('risk_model_factor_exposure.csv', index_col=['ID'])
df_factor_exp = df_factor_exp.pivot(columns=['FACTOR'],values=['DATA'])
df_factor_exp.columns=df_factor_exp.columns.droplevel(0)
len(df_factor_exp.columns)

206

In [12]:
df_factor_zoo = pd.read_csv('risk_model_risk_factors.csv', index_col=['FACTOR'])
df_factor_zoo = df_factor_zoo[['HIERARCHY1','HIERARCHY2','NAME']]
set(df_factor_zoo.HIERARCHY1)

{'Risk Type : Equity',
 'Risk Type : FX',
 'Risk Type : Interest Rate',
 'Risk Type : Issuer Credit',
 'Risk Type : Vega'}

In [13]:
df_factor_cov = pd.read_csv('risk_model_factor_covariance.csv')
key1 = "FACTOR_1"
key2 = "FACTOR_2"
value = "DATA"
dict_factor_cov = {
    (df_factor_cov.loc[i,key1],df_factor_cov.loc[i,key2]) : 
    df_factor_cov.loc[i,value] for i in range(len(df_factor_cov))
    }
dict_factor_cov[(2,2)]

0.025747948

In [14]:
df_spec_cov = pd.read_csv('risk_model_specific_variance_covariance.csv')
key1 = "ID_1"
key2 = "ID_2"
value = "DATA"
dict_spec_cov = {
    (df_spec_cov.loc[i,key1],df_spec_cov.loc[i,key2]) : 
    df_spec_cov.loc[i,value] for i in range(len(df_spec_cov))
    }
dict_spec_cov[("AT0000A24UY3","XS2154348424")]

0.000328514

Q1: portfolio ex-ante risk

In [15]:
def get_portfolio_risk_exposures(df_wgt, df_risk_exp):
    m_exp = np.matmul(
        np.matrix(df_wgt).T, 
        np.matrix(df_risk_exp.fillna(0).loc[list(df_wgt.index),:])
        )
    return pd.DataFrame(m_exp, columns=df_risk_exp.columns)

In [16]:
df_port_exp = get_portfolio_risk_exposures(df_port_wgt, df_factor_exp)
df_mkt_exp = get_portfolio_risk_exposures(df_mkt_wgt, df_factor_exp)

In [17]:
def get_portfolio_ex_ante_risk(df_exp, dict_factor_cov):
    df_risk_contrib = df_exp * 0.0
    for k in dict_factor_cov.keys():
        tmp_risk = df_exp[k[0]] * dict_factor_cov[k] * df_exp[k[1]]
        df_risk_contrib[k[0]] += tmp_risk
    df_risk_contrib.index = ["Risk"]
    return df_risk_contrib

In [18]:
df_risk_contrib = get_portfolio_ex_ante_risk(df_port_exp, dict_factor_cov)
risk = df_risk_contrib.sum(axis=1)
risk

Risk    0.00989
dtype: float64

In [ ]:
def get_specific_ex_ante_risk(df_wgt, dict_spec_cov):
    spec_risk = df_wgt * 0.0
    for k in dict_spec_cov.keys():
        try:
            tmp_risk = df_wgt[k[0]] * dict_spec_cov[k] * df_wgt[k[1]]
            spec_risk[k[0]] += tmp_risk
        except:
            pass #security not in portfolio
    spec_risk.index = ["Risk"]
    return spec_risk

In [ ]:
spec_risk_contrib = get_specific_ex_ante_risk(df_port_wgt, dict_spec_cov)
spec_risk = spec_risk_contrib.sum(axis=1)
spec_risk

Q2: fully hedged

In [19]:
def get_portfolio_risk_exp_hedged(df_wgt, df_risk_exp, df_factor_zoo):
    df_exp = get_portfolio_risk_exposures(df_wgt, df_risk_exp)
    credit_risk = list(
        df_factor_zoo[df_factor_zoo.HIERARCHY1=='Risk Type : Issuer Credit'].index
        )
    for k in df_exp.columns:
        if k not in credit_risk:
            df_exp[k] = 0.0
    return df_exp

In [20]:
df_port_exp_hdg = get_portfolio_risk_exp_hedged(df_port_wgt, df_factor_exp, df_factor_zoo)
df_mkt_exp_hdg = get_portfolio_risk_exp_hedged(df_mkt_wgt, df_factor_exp, df_factor_zoo)

In [21]:
df_risk_contrib_hdg = get_portfolio_ex_ante_risk(df_port_exp_hdg, dict_factor_cov)
risk = df_risk_contrib_hdg.sum(axis=1)
risk

Risk    0.00166
dtype: float64

Q3: top 3 contributors

In [22]:
top5_risks = df_risk_contrib_hdg.transpose().sort_values(by='Risk',ascending=False).head(5)
top5_risks.join(df_factor_zoo)

Risk                 HIERARCHY1      HIERARCHY2  \
FACTOR                                                        
71      0.000325  Risk Type : Issuer Credit  Credit Quality   
74      0.000247  Risk Type : Issuer Credit  Credit Quality   
73      0.000202  Risk Type : Issuer Credit  Credit Quality   
55      0.000193  Risk Type : Issuer Credit    Credit Style   
76      0.000161  Risk Type : Issuer Credit    Credit Style   

                                   NAME  
FACTOR                                   
71      Credit Quality - USD Quality Q1  
74      Credit Quality - USD Quality Q4  
73      Credit Quality - USD Quality Q3  
55              Credit Style - Momentum  
76                 Credit Style - Value

Q4: vs market portfolio

L/S credit portfolio is less likely to lose more than the market portfolio in market drawdowns.
- market portfolio has high risk than the l/s credit portfolio
- market portfolio has significant beta risk to the markets, while l/s credit portfolio is only slightly skewed in credit quality.

In [23]:
df_risk_contrib_hdg_mkt = get_portfolio_ex_ante_risk(df_mkt_exp_hdg, dict_factor_cov)
risk = df_risk_contrib_hdg_mkt.sum(axis=1)
risk

Risk    0.002352
dtype: float64

In [24]:
top5_risks_mkt = df_risk_contrib_hdg_mkt.transpose().sort_values(by='Risk',ascending=False).head(5)
top5_risks_mkt.join(df_factor_zoo)

Risk                 HIERARCHY1                 HIERARCHY2  \
FACTOR                                                                   
33      0.001261  Risk Type : Issuer Credit    Credit Market Intercept   
5       0.000403  Risk Type : Issuer Credit               Credit Style   
70      0.000391  Risk Type : Issuer Credit  Credit Currency Intercept   
69      0.000116  Risk Type : Issuer Credit             Credit Country   
66      0.000085  Risk Type : Issuer Credit               Credit Style   

                                                     NAME  
FACTOR                                                     
33      Credit Market Intercept - Global Market Intercept  
5                                     Credit Style - Beta  
70              Credit Currency Intercept - USD Intercept  
69                         Credit Country - United States  
66                               Credit Style - Steepness

Q5: correlation reduction

- further neutralize at various buckets, e.g. sector, industry, country levels, etc.
- more alpha research on issuer specific drivers, e.g. governance, event-driven, etc.
- apply beta timing using macro or market regime classification
- focus on liquidity risk, as market drawdowns typically triggering liquidity events
- establish factor risk lmits to control factor-based risks